In [1]:
import tensorflow as tf
import os
import cv2
from ops import *
import pandas as pd

In [2]:
import math
import pandas as pd
import numpy as np

class Dataset:
    def __init__(self):
        '''
        Read the data and split the values
        '''
        data = pd.read_csv('mnist_train.csv')
        self.data = np.array(data.iloc[:2,1:],dtype=np.float32) # Train only on 7000 images
        del data
        
        self.data = self.data.reshape([-1,28,28,1])
        
        self.normalize()
    
    def normalize(self):
        # self.data+=np.random.randn(self.data.shape[0],self.data.shape[1],self.data.shape[2],self.data.shape[3])
        self.data = (self.data/127.5) - 1.0
    
    def denormalize(self, batch):
        batch = (batch + 1.0)*127.5
        return batch
    
    def save_batch(self,batch,idx):
        print(np.min(batch[0]))
        print(np.max(batch[0]))
        
        batch = self.denormalize(batch)
        batch = batch.astype(np.uint8)
        
        # Save the images
        save_dir = 'Generated-Images'
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        
        if not os.path.exists(save_dir + '/' + str(idx)):
            os.mkdir(save_dir + '/' + str(idx))
        
        save_dir = save_dir + '/' + str(idx) + '/'
        
        print(np.min(batch[0]))
        print(np.max(batch[0]))
        
        for i in range(batch.shape[0]):
            cv2.imwrite(save_dir + str(i) + '.png',batch[i])
        
        
    def get_minibatches(self, batch_size):
        perms = np.random.permutation(self.data.shape[0])
        self.data = self.data[perms]
        
        num_batches = int(self.data.shape[0]/batch_size)
        
        batches = []
        for idx in range(num_batches):
            start_idx = idx*batch_size
            end_idx = (idx+1)*batch_size
            
            batches.append(self.data[start_idx:end_idx,:])
        
        frac_batches = self.data.shape[0]/batch_size
        
        if math.floor(frac_batches) != math.ceil(frac_batches):
            start_idx = num_batches*batch_size
            batches.append(self.data[start_idx:,:])
        
        return batches

In [3]:
def Discriminator(x, is_train,reuse,name):
        with tf.variable_scope(name,reuse=reuse):
            ###  First Conv Block ###
            x = pad(x,[[0,0],[1,1],[1,1],[0,0]])
            x = conv2d(x,3,32,1,padding='VALID',name='conv1')
            x = lrelu(x,name='a1')            
            x = avg_pool(x,2,2,padding='VALID',name='pool1')
              
            ### Second Conv Block ###
            x = bn(x,is_train=is_train,name='bn2')
            x = pad(x,[[0,0],[2,2],[2,2],[0,0]])
            x = conv2d(x,4,64,1,padding='VALID',name='conv2')
            x = lrelu(x,name='a2')            
            x = avg_pool(x,2,2,padding='VALID',name='pool2')
            
            ### Second Conv Block ###
            x = bn(x,is_train=is_train,name='bn2_1')
            x = pad(x,[[0,0],[2,2],[2,2],[0,0]])
            x = conv2d(x,4,128,1,padding='VALID',name='conv2_1')
            x = lrelu(x,name='a2_1')
            # x = max_pool(x,2,2,padding='VALID',name='pool2_1')
            x = avg_pool(x,2,2,padding='VALID',name='pool2_1')
            
            ### Flatten ###
            x = flatten(x)
            x = bn(x,is_train=is_train,name='bn3')
            x = dense(x,4*4*64,name='dense3')
            x = lrelu(x,name='a3')

            x = bn(x,is_train=is_train,name='bn4')
            x = dense(x,4*4*32,name='dense4')
            x = lrelu(x,name='a4')
            
            x = bn(x,is_train=is_train,name='bn4_1')
            x = dense(x,4*4*16,name='dense4_1')
            x = lrelu(x,name='a4_1')
            
            x = bn(x,is_train=is_train,name='bn5')
            x = dense(x,1,name='dense5')

            out = x
            prob = tf.nn.sigmoid(out,name='Prob')
            
            return prob    

def Generator(x,is_train,noise_dim,name='Generator'):
        with tf.variable_scope(name):
            x = dense(x,7*7*8,name='dense1')
            x = relu(x,name='a1')
            
            x = bn(x,is_train=is_train,name='bn1_1')
            x = dense(x,3*3*16,name='dense1_1')
            x = lrelu(x,name='a1_1')
            
            x = bn(x,is_train=is_train,name='bn2')
            x = dense(x,3*3*32,name='dense2')
            x = lrelu(x,name='a2')
            
            x = bn(x,is_train=is_train,name='bn2_1')
            x = dense(x,3*3*64,name='dense2_1')
            x = lrelu(x,name='a2_1')
            
            x = bn(x,is_train=is_train,name='bn2_2')
            x = dense(x,3*3*128,name='dense2_2')
            x = lrelu(x,name='a2_2')
            
            x = tf.reshape(x,[-1,3,3,128])
            x = bn(x,is_train=is_train,name='bn3_0')
            x = D_conv2d(x,3,64,2,padding='VALID',name='dconv3_0')
            x = lrelu(x,name='a3_0')
            
            x = bn(x,is_train=is_train,name='bn3')
            x = D_conv2d(x,4,64,2,name='dconv3')
            x = lrelu(x,name='a3')

            x = D_conv2d(x,4,1,2,name='dconv3_1')
            x = tanh(x,name='a3_1')
            
            out = x
            return out      

In [4]:
class DCGAN:
    def __init__(self,noise_dim,batch_size,d_lr=1e-3,g_lr=1e-3,d_iter=1,g_iter=1,\
                 num_epochs=30000,W=28,H=28,C=1,summary_path='tensorboard',name='DCGAN'):
        self.noise_dim = noise_dim
        self.batch_size = batch_size
        self.d_lr = d_lr
        self.g_lr = g_lr
        self.d_iter = d_iter
        self.g_iter = g_iter
        self.num_epochs = num_epochs
        self.name = name
        
        # Image Parameters
        self.W = W
        self.H = H
        self.C = C
        
        # Dataset
        self.dataset = Dataset()
        
        self.summary_path = summary_path
        if not os.path.exists(self.summary_path):
            os.mkdir(self.summary_path)

        self.writer = tf.summary.FileWriter(self.summary_path)
        self.build_model()
        
    def log_variables(self):
        '''
        Summaries For Variables to add in tensorboard for visualisation
        '''
        tf.summary.scalar('Discriminator_Loss',self.dloss_)
        tf.summary.scalar('Generator_Loss',self.gloss_)
        
        # Add the gradients to the summary 
#         d_grads = self.D_optimiser.compute_gradients(self.D_loss)
#         g_grads = self.G_optimiser.compute_gradients(self.G_loss)
                    
#         for index,grads in enumerate(d_grads):
#             tf.summary.histogram("{}-D_grad".format(d_grads[index][1].name), d_grads[index])
                            
#         for index,grads in enumerate(g_grads):
#             tf.summary.histogram("{}-G_grad".format(g_grads[index][1].name), g_grads[index])
        
        self.merged = tf.summary.merge_all()
        
    
    def build_model(self):
        tf.reset_default_graph()
        with tf.variable_scope(self.name):
            with tf.name_scope('Placeholders'):
                self.inputs = tf.placeholder(shape=[None,self.W,self.H,self.C],name='inputs',dtype=tf.float32)
                self.is_train = tf.placeholder(name='is_train',dtype=tf.bool)
                self.noise = tf.placeholder(shape=[None,self.noise_dim],name='noise',dtype=tf.float32)
                
                # Placeholders for tensorboard visualisation of losses
                self.dloss_ = tf.placeholder(name='dloss_',dtype=tf.float32)
                self.gloss_ = tf.placeholder(name='gloss_',dtype=tf.float32)

            # Real logits
            self.D_real = Discriminator(self.inputs,self.is_train,reuse=False,name='Dis')
            
            # Fake images
            self.fake_images = Generator(self.noise,self.is_train,self.noise_dim,name='Gen')
            
            # Fake logits
            self.D_fake = Discriminator(self.fake_images,self.is_train,reuse=True,name='Dis')

            # Define the losses
            # D_real must tend to 1.0 and D_fake must tend to 0.0
            # self.D_loss = tf.reduce_mean(-tf.log(self.D_real) - tf.log(1.0 - self.D_fake))
            # D_fake must tend to 1.0
            # self.G_loss = tf.reduce_mean(-tf.log(self.D_fake))
            # self.D_loss = tf.reduce_mean(tf.squared_difference(self.D_real,1.0) + tf.square(self.D_fake))
            # self.G_loss = tf.reduce_mean(tf.squared_difference(self.D_fake,1.0))
            self.D_loss = tf.reduce_mean(self.D_fake) - tf.reduce_mean(self.D_real)
            self.G_loss = -tf.reduce_mean(self.D_fake)
            
            # Seperate the variables
            var = tf.trainable_variables()
            self.d_vars = [v for v in var if 'Dis' in v.name]
            self.g_vars = [v for v in var if 'Gen' in v.name]
            
            self.d_clip = [v.assign(tf.clip_by_value(v, -0.1, 0.1)) for v in self.d_vars]
            
            # Define the optimizers
            self.D_optimiser = tf.train.AdamOptimizer(self.d_lr)
            self.G_optimiser = tf.train.AdamOptimizer(self.g_lr)
            
            self.D_optim = self.D_optimiser.minimize(self.D_loss,var_list=self.d_vars)
            self.G_optim = self.G_optimiser.minimize(self.G_loss,var_list=self.g_vars)

            self.sess = tf.Session()
            self.sess.run(tf.global_variables_initializer())
            self.sess.run(tf.local_variables_initializer())
                        
            self.log_variables()
            
            # Write the graph summary to disk
            self.writer.add_graph(self.sess.graph)
            
    
    def train(self):
        for epoch in range(self.num_epochs):
            dloss_epoch = 0.0
            gloss_epoch = 0.0
            batch_count = 0
            
            for batch in self.dataset.get_minibatches(self.batch_size):
                saved = False
                
                # batch corresponds to the real image
                # Sample some noise
                noise = np.random.uniform(-1.0,1.0,size=[batch.shape[0],self.noise_dim]).astype(np.float32)
                
                # Train the discriminator
                for _ in range(self.d_iter):
                    self.sess.run(self.d_clip)
                    _ = self.sess.run([self.D_optim],feed_dict={self.inputs:batch,self.noise:noise,self.is_train:True})
                
                # Train the generator
                for _ in range(self.g_iter):
                    _ = self.sess.run([self.G_optim],feed_dict={self.inputs:batch,self.noise:noise,self.is_train:True})
                
                dloss_batch = self.sess.run(self.D_loss,feed_dict={self.inputs:batch,self.noise:noise,self.is_train:True})
                gloss_batch = self.sess.run(self.G_loss,feed_dict={self.inputs:batch,self.noise:noise,self.is_train:True})
                
                # Update statistics
                dloss_epoch+=dloss_batch
                gloss_epoch+=gloss_batch
                batch_count+=1
                
                # Generate images every 100 epochs
                if epoch%100 == 0:
                    if saved is False:
                        generated_batch = self.sess.run(self.fake_images,feed_dict={self.noise:noise,self.is_train:False})
                        generated_batch = np.array(generated_batch)
                        self.dataset.save_batch(generated_batch,epoch)
                        saved = True
                        
                    drl,dfk = self.sess.run([self.D_real,self.D_fake],feed_dict={self.inputs:batch,self.noise:noise,self.is_train:True})
                    print('D Real : {}, D Fake : {}'.format(drl,dfk))
                
                print('Epoch[{}/{}] - Batch : {}'.format(epoch,self.num_epochs,batch_count))
            
            # Not logging gradients at the moment...
            summary = self.sess.run(self.merged,feed_dict={self.dloss_:dloss_epoch,self.gloss_:gloss_epoch,\
                                                                       self.inputs:batch,self.noise:noise,self.is_train:True})
            self.writer.add_summary(summary,epoch)
            self.writer.flush()
            
            dloss_epoch/=(1.0*batch_count)
            gloss_epoch/=(1.0*batch_count)
            
            print('D Loss : {}, G Loss : {}'.format(dloss_epoch,gloss_epoch))

In [6]:
dcgan = DCGAN(10,25)

In [7]:
dcgan.train()
# tf.trainable_variables()

-0.0014131936
0.005056422
127
128
D Real : [[0.50396466]
 [0.4952981 ]], D Fake : [[0.49575213]
 [0.50351065]]
Epoch[0/30000] - Batch : 1
D Loss : 2.9802322387695312e-08, G Loss : -0.499631404876709
Epoch[1/30000] - Batch : 1
D Loss : -4.172325134277344e-07, G Loss : -0.4990231394767761
Epoch[2/30000] - Batch : 1
D Loss : -8.940696716308594e-07, G Loss : -0.4982718825340271
Epoch[3/30000] - Batch : 1
D Loss : -5.066394805908203e-06, G Loss : -0.4975033104419708
Epoch[4/30000] - Batch : 1
D Loss : -1.9788742065429688e-05, G Loss : -0.4967110753059387
Epoch[5/30000] - Batch : 1
D Loss : -5.245208740234375e-05, G Loss : -0.49587374925613403
Epoch[6/30000] - Batch : 1
D Loss : -9.629130363464355e-05, G Loss : -0.4948854148387909
Epoch[7/30000] - Batch : 1
D Loss : -0.00012600421905517578, G Loss : -0.4938092827796936
Epoch[8/30000] - Batch : 1
D Loss : -0.00018414855003356934, G Loss : -0.4925737977027893
Epoch[9/30000] - Batch : 1
D Loss : -0.00023114681243896484, G Loss : -0.491229206323

Epoch[91/30000] - Batch : 1
D Loss : -0.14774870872497559, G Loss : -0.13544410467147827
Epoch[92/30000] - Batch : 1
D Loss : -0.06167669594287872, G Loss : -0.22474460303783417
Epoch[93/30000] - Batch : 1
D Loss : -0.12976160645484924, G Loss : -0.15505850315093994
Epoch[94/30000] - Batch : 1
D Loss : -0.14676792919635773, G Loss : -0.14323993027210236
Epoch[95/30000] - Batch : 1
D Loss : -0.07579691708087921, G Loss : -0.2163187712430954
Epoch[96/30000] - Batch : 1
D Loss : -0.09046304225921631, G Loss : -0.19469201564788818
Epoch[97/30000] - Batch : 1
D Loss : -0.07351845502853394, G Loss : -0.19413766264915466
Epoch[98/30000] - Batch : 1
D Loss : -0.11391332745552063, G Loss : -0.1601496934890747
Epoch[99/30000] - Batch : 1
D Loss : -0.1204550713300705, G Loss : -0.15622667968273163
-0.3594756
0.9928111
81
254
D Real : [[0.02855554]
 [0.5156024 ]], D Fake : [[0.35009348]
 [0.05489494]]
Epoch[100/30000] - Batch : 1
D Loss : -0.06958474218845367, G Loss : -0.20249421894550323
Epoch[1

Epoch[183/30000] - Batch : 1
D Loss : -0.10082001984119415, G Loss : -0.1787712126970291
Epoch[184/30000] - Batch : 1
D Loss : -0.08989641070365906, G Loss : -0.17714691162109375
Epoch[185/30000] - Batch : 1
D Loss : -0.09137424826622009, G Loss : -0.1778411567211151
Epoch[186/30000] - Batch : 1
D Loss : -0.08863890171051025, G Loss : -0.18273374438285828
Epoch[187/30000] - Batch : 1
D Loss : -0.0933539867401123, G Loss : -0.17941460013389587
Epoch[188/30000] - Batch : 1
D Loss : -0.09381020069122314, G Loss : -0.18018370866775513
Epoch[189/30000] - Batch : 1
D Loss : -0.09378644824028015, G Loss : -0.18155908584594727
Epoch[190/30000] - Batch : 1
D Loss : -0.09532412886619568, G Loss : -0.18138423562049866
Epoch[191/30000] - Batch : 1
D Loss : -0.09784603118896484, G Loss : -0.18198400735855103
Epoch[192/30000] - Batch : 1
D Loss : -0.06811545789241791, G Loss : -0.2104954868555069
Epoch[193/30000] - Batch : 1
D Loss : -0.09688693284988403, G Loss : -0.18262994289398193
Epoch[194/3000

Epoch[274/30000] - Batch : 1
D Loss : -0.10712999105453491, G Loss : -0.17513194680213928
Epoch[275/30000] - Batch : 1
D Loss : -0.10869626700878143, G Loss : -0.1738429218530655
Epoch[276/30000] - Batch : 1
D Loss : -0.10933510959148407, G Loss : -0.17366211116313934
Epoch[277/30000] - Batch : 1
D Loss : -0.10969701409339905, G Loss : -0.17351844906806946
Epoch[278/30000] - Batch : 1
D Loss : -0.10939060151576996, G Loss : -0.17400427162647247
Epoch[279/30000] - Batch : 1
D Loss : -0.11039102077484131, G Loss : -0.1731680929660797
Epoch[280/30000] - Batch : 1
D Loss : -0.11072327196598053, G Loss : -0.1729937344789505
Epoch[281/30000] - Batch : 1
D Loss : -0.10264983773231506, G Loss : -0.18116894364356995
Epoch[282/30000] - Batch : 1
D Loss : -0.11135557293891907, G Loss : -0.17259365320205688
Epoch[283/30000] - Batch : 1
D Loss : -0.10887756943702698, G Loss : -0.17512813210487366
Epoch[284/30000] - Batch : 1
D Loss : -0.11194616556167603, G Loss : -0.17215341329574585
Epoch[285/300

Epoch[365/30000] - Batch : 1
D Loss : -0.1408068686723709, G Loss : -0.16104356944561005
Epoch[366/30000] - Batch : 1
D Loss : -0.1419193148612976, G Loss : -0.16020885109901428
Epoch[367/30000] - Batch : 1
D Loss : -0.14233753085136414, G Loss : -0.1600458323955536
Epoch[368/30000] - Batch : 1
D Loss : -0.14273804426193237, G Loss : -0.15991294384002686
Epoch[369/30000] - Batch : 1
D Loss : -0.14214327931404114, G Loss : -0.16070795059204102
Epoch[370/30000] - Batch : 1
D Loss : -0.1435094028711319, G Loss : -0.15952961146831512
Epoch[371/30000] - Batch : 1
D Loss : -0.14362239837646484, G Loss : -0.15962207317352295
Epoch[372/30000] - Batch : 1
D Loss : -0.1421346664428711, G Loss : -0.161260724067688
Epoch[373/30000] - Batch : 1
D Loss : -0.1427638828754425, G Loss : -0.16074120998382568
Epoch[374/30000] - Batch : 1
D Loss : -0.1450522243976593, G Loss : -0.15834471583366394
Epoch[375/30000] - Batch : 1
D Loss : -0.14520666003227234, G Loss : -0.15817612409591675
Epoch[376/30000] - 

...

Epoch[921/30000] - Batch : 1
D Loss : -0.21037395298480988, G Loss : -0.13285650312900543
Epoch[922/30000] - Batch : 1
D Loss : -0.2103600800037384, G Loss : -0.13288262486457825
Epoch[923/30000] - Batch : 1
D Loss : -0.2102697193622589, G Loss : -0.13299047946929932
Epoch[924/30000] - Batch : 1
D Loss : -0.2103385031223297, G Loss : -0.13294249773025513
Epoch[925/30000] - Batch : 1
D Loss : -0.21042275428771973, G Loss : -0.13288196921348572
Epoch[926/30000] - Batch : 1
D Loss : -0.210443913936615, G Loss : -0.13288918137550354
Epoch[927/30000] - Batch : 1
D Loss : -0.21026557683944702, G Loss : -0.13309821486473083
Epoch[928/30000] - Batch : 1
D Loss : -0.21047545969486237, G Loss : -0.132918581366539
Epoch[929/30000] - Batch : 1
D Loss : -0.21041420102119446, G Loss : -0.13300484418869019
Epoch[930/30000] - Batch : 1
D Loss : -0.21041671931743622, G Loss : -0.13302300870418549
Epoch[931/30000] - Batch : 1
D Loss : -0.21043160557746887, G Loss : -0.13301962614059448
Epoch[932/30000] 

Epoch[1014/30000] - Batch : 1
D Loss : -0.21058489382266998, G Loss : -0.1329837292432785
Epoch[1015/30000] - Batch : 1
D Loss : -0.21037936210632324, G Loss : -0.13316619396209717
Epoch[1016/30000] - Batch : 1
D Loss : -0.21089661121368408, G Loss : -0.13263556361198425
Epoch[1017/30000] - Batch : 1
D Loss : -0.21086221933364868, G Loss : -0.13265365362167358
Epoch[1018/30000] - Batch : 1
D Loss : -0.20887711644172668, G Loss : -0.13462796807289124
Epoch[1019/30000] - Batch : 1
D Loss : -0.21085888147354126, G Loss : -0.13264325261116028
Epoch[1020/30000] - Batch : 1
D Loss : -0.21088290214538574, G Loss : -0.13262662291526794
Epoch[1021/30000] - Batch : 1
D Loss : -0.21085113286972046, G Loss : -0.13267210125923157
Epoch[1022/30000] - Batch : 1
D Loss : -0.2108837068080902, G Loss : -0.13266000151634216
Epoch[1023/30000] - Batch : 1
D Loss : -0.2108193188905716, G Loss : -0.13274766504764557
Epoch[1024/30000] - Batch : 1
D Loss : -0.21077968180179596, G Loss : -0.1327819973230362
Epo

Epoch[1106/30000] - Batch : 1
D Loss : -0.21084585785865784, G Loss : -0.13270312547683716
Epoch[1107/30000] - Batch : 1
D Loss : -0.2108326256275177, G Loss : -0.1326993703842163
Epoch[1108/30000] - Batch : 1
D Loss : -0.21082031726837158, G Loss : -0.13273215293884277
Epoch[1109/30000] - Batch : 1
D Loss : -0.21084997057914734, G Loss : -0.13271057605743408
Epoch[1110/30000] - Batch : 1
D Loss : -0.21085502207279205, G Loss : -0.13271479308605194
Epoch[1111/30000] - Batch : 1
D Loss : -0.21085192263126373, G Loss : -0.13272221386432648
Epoch[1112/30000] - Batch : 1
D Loss : -0.21083961427211761, G Loss : -0.13274739682674408
Epoch[1113/30000] - Batch : 1
D Loss : -0.21075350046157837, G Loss : -0.13283994793891907
Epoch[1114/30000] - Batch : 1
D Loss : -0.21085509657859802, G Loss : -0.1327463686466217
Epoch[1115/30000] - Batch : 1
D Loss : -0.210820272564888, G Loss : -0.13278917968273163
Epoch[1116/30000] - Batch : 1
D Loss : -0.2107926309108734, G Loss : -0.13282287120819092
Epoch

Epoch[1197/30000] - Batch : 1
D Loss : -0.20834240317344666, G Loss : -0.13427647948265076
Epoch[1198/30000] - Batch : 1
D Loss : -0.20867864787578583, G Loss : -0.13377465307712555
Epoch[1199/30000] - Batch : 1
D Loss : -0.20879340171813965, G Loss : -0.1335083246231079
-0.9998979
0.99999917
0
254
D Real : [[0.01137249]
 [0.67287177]], D Fake : [[0.14974138]
 [0.11844002]]
Epoch[1200/30000] - Batch : 1
D Loss : -0.20803144574165344, G Loss : -0.13409069180488586
Epoch[1201/30000] - Batch : 1
D Loss : -0.20881609618663788, G Loss : -0.13317279517650604
Epoch[1202/30000] - Batch : 1
D Loss : -0.208769291639328, G Loss : -0.13313087821006775
Epoch[1203/30000] - Batch : 1
D Loss : -0.2088589370250702, G Loss : -0.13300389051437378
Epoch[1204/30000] - Batch : 1
D Loss : -0.2088182270526886, G Loss : -0.1330569088459015
Epoch[1205/30000] - Batch : 1
D Loss : -0.2088945508003235, G Loss : -0.1330384612083435
Epoch[1206/30000] - Batch : 1
D Loss : -0.20889848470687866, G Loss : -0.13313072919

KeyboardInterrupt: 

In [11]:
# print(dcgan.d_vars)
# print(dcgan.g_vars)
# dataset = Dataset()

In [ ]:
# for batch in dataset.get_minibatches(32):
#     print(batch.shape)
#     print(np.max(batch))
#     print(np.min(batch))
#     batch = dataset.denormalize(batch)

#     batch = batch.astype(np.uint8)
#     print(batch.shape)
#     print(np.max(batch))
#     print(np.min(batch))
    
#     import matplotlib.pyplot as plt
#     plt.figure()
#     plt.imshow(batch[5].reshape([28,28]))
#     cv2.imwrite('test.png',batch[5])
    
#     break
tf.truncated_normal_initializer()

In [ ]:
'''
Did not work with mean squared error
Use lrelu for G and D - avoids sparse gradients
Use average pooling for D - avoids sparse gradients
'''